In [1]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos


In [2]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_a1_WALLET = WalletConfig("a1", "pin")

TRADER_a2_WALLET = WalletConfig("a2", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_3_WALLET = WalletConfig("T3", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_2_WALLET, TRADER_3_WALLET, TRADER_a1_WALLET , TRADER_a2_WALLET, TERMINATE_WALLET]


In [3]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False, transactions_per_block=10)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/tmpdqujz63a
INFO:vega_sim.null_service:Launching GraphQL node at port 55091
INFO:vega_sim.null_service:Launching Console at port 55097
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [4]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [5]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)
vega.wait_fn(2)

tdai_id = vega.find_asset_id(symbol="tDAI")

mint_amount = {MM_WALLET.name: 2000000, TRADER_a1_WALLET.name: 2000000, TRADER_a2_WALLET.name: 2000000, TRADER_2_WALLET.name: 2000, TRADER_3_WALLET.name: 90}

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=mint_amount[MM_WALLET.name]
)
vega.mint(
    TRADER_a1_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a1_WALLET.name]
)
vega.mint(
    TRADER_a2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a2_WALLET.name]
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_2_WALLET.name]
)
vega.mint(
    TRADER_3_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_3_WALLET.name]
)
vega.wait_for_total_catchup()


In [6]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=0,
        market_decimals=0,
        risk_aversion=0.000001,
        tau=0.1,
        sigma=1,
        
)

market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [7]:
vega.update_network_parameter(MM_WALLET.name, "market.margin.scalingFactors", "{\"search_level\": 1.5, \"initial_margin\": 2, \"collateral_release\": 3}")

In [8]:
# submit liquidity
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=100000,
    fee=0.001,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=55,
    delta_sell=55,
    is_amendment=False
)

vega.forward('1s')
vega.forward('1s')

In [9]:
vega.party_account(MM_WALLET.name, market_id=market_id, asset_id=tdai_id)

AccountData(general=1900000.0, margin=0.0, bond=100000.0)

In [10]:
vega.data_node_graphql_port

55091

In [11]:
# Post GTC limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=5,
    price=5,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=1000,
    wait=True,
)

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=10,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=10,
    wait=True,
)


In [12]:
# Check order status/ market state

for wallet in wallets:
    if wallet.name in mint_amount:
        general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
        print(wallet.name + " General = "+str(general))
        print(wallet.name + " Margin  = "+str(margin))
        print(wallet.name + " Bond    = "+str(bond))
        print(vega.margin_levels(wallet.name))
        print(general+margin+bond - mint_amount[wallet.name])
        # print(vega.order_status(order_id=buy_order_id))

print(vega.market_data(market_id=market_id))

Using function with raw data from data-node VegaService.market_data. Be wary if prices/positions are not converted from int form


mm General = 1900000.0
mm Margin  = 0.0
mm Bond    = 100000.0
[]
0.0
T2 General = 2000.0
T2 Margin  = 0
T2 Bond    = 0
[]
0.0
T3 General = 90.0
T3 Margin  = 0
T3 Bond    = 0
[]
0.0
a1 General = 1857723.85632
a1 Margin  = 142276.14368
a1 Bond    = 0
[MarginLevels(maintenance_margin=36636.107, search_level=54954.1605, initial_margin=73272.214, collateral_release_level=252184.46468, party_id='64f4638929c274042f8a83c09f16f7158c511c852719cd94dbc8a13bcaa20cf2', market_id='8418baba53a18afa8f3cdd815ac8312003553c7368d8fc97df451ee595aa7707', asset='b697f148ac152c8468f1f7d13cf1f5ff880456e85777b1ff3cb3b6259925b88f', timestamp=1638185433259066000)]
0.0
a2 General = 1991952.6815
a2 Margin  = 8047.3185
a2 Bond    = 0
[MarginLevels(maintenance_margin=220.20023, search_level=330.30034, initial_margin=440.40046, collateral_release_level=8707.91919, party_id='c95412b66fceafc5dc018471b00529fa33de5f3d902d13d855b8d03b06e7ed88', market_id='8418baba53a18afa8f3cdd815ac8312003553c7368d8fc97df451ee595aa7707', as

In [13]:
new_buy_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=40,
    price=10,
    wait=True,
)
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 1128322.2114
mm Margin  = 771677.7886
mm Bond    = 100000.0
T2 General = 1359.41742
T2 Margin  = 640.58258
T2 Bond    = 0
T3 General = 90.0
T3 Margin  = 0
T3 Bond    = 0
a1 General = 1978777.23856
a1 Margin  = 21222.76144
a1 Bond    = 0
a2 General = 1999759.78152
a2 Margin  = 240.21848
a2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [ ]:
# Amend order
vega.amend_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_id=buy_order_id,
    price=75,
    volume_delta=2,
)



NameError: name 'TRADER_1_WALLET' is not defined

In [15]:
# Check order status/ market state after amend
print(vega.order_status(order_id=new_buy_order_id))

print(vega.market_data(market_id=market_id))

Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.market_data. Be wary if prices/positions are not converted from int form


id: "82cc83ce0357d1b3ea49bfda49f95e61ee6dc25967041acd43a317654f8cd658"
market_id: "f325216c8e1a6207bcaa929d9030ad5680ad693c8eaf18053d3b554a99d7cf38"
party_id: "8b9e94d400e33a7b353062bca685ff173c8a5a1e28e1d7188b8fd5270d607e3c"
side: SIDE_BUY
price: "10"
size: 40
remaining: 40
time_in_force: TIME_IN_FORCE_GTC
type: TYPE_LIMIT
created_at: 1638185445259066000
status: STATUS_ACTIVE
reference: "8b9e94d400e33a7b353062bca685ff173c8a5a1e28e1d7188b8fd5270d607e3c-0cc6f2de-29e7-4054-ac51-bfac9fa3b1ff"
version: 1

mark_price: "10"
best_bid_price: "10"
best_bid_volume: 40
best_offer_price: "1000"
best_offer_volume: 12
best_static_bid_price: "10"
best_static_bid_volume: 40
best_static_offer_price: "1000"
best_static_offer_volume: 10
mid_price: "505"
static_mid_price: "505"
market: "f325216c8e1a6207bcaa929d9030ad5680ad693c8eaf18053d3b554a99d7cf38"
timestamp: 1638185447259066000
open_interest: 10
indicative_price: "0"
market_trading_mode: TRADING_MODE_CONTINUOUS
target_stake: "35569000"
supplied_stake: